# Convert Freebase IDs to Wikidata IDs

## Prerequisites

Deploy the wikidata endpoint locally for fast queries. Please follow [qEndpoint - wikidata](https://github.com/the-qa-company/qEndpoint#qacompanyqendpoint-wikidata) to set up the endpoint.

In enssence, you only need to run the following command:

`# docker run -p 1234:1234 --name qendpoint-wikidata qacompany/qendpoint-wikidata`

## Part 1: Convert Entity IDs

In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper(
    "http://localhost:1234/api/endpoint/sparql"
)
sparql.setReturnFormat(JSON)

def queryFreebaseId(entity):
    sparql.setQuery(f"""
        SELECT DISTINCT ?fid WHERE {{
            <http://www.wikidata.org/entity/{entity}> <http://www.wikidata.org/prop/direct/P646> ?fid
        }}
        LIMIT 1
        """)
    try:
        ret = sparql.queryAndConvert()        
        r = ret["results"]["bindings"]
        if len(r) > 0:
            fid = r[0]["fid"]["value"]
            return fid
        return None
    except Exception as e:
        print(e)
        return None

In [2]:
import pickle

with open('id2entity.pkl', 'rb') as f:
    id2entity = pickle.load(f)

In [ ]:
from tqdm import tqdm

fids = map(queryFreebaseId, tqdm(id2entity))
fid2qid = {fid: qid for fid, qid in zip(fids, id2entity) if fid is not None}
print("Found:", len(fid2qid))
print(f"Not found: {len(id2entity) - len(fid2qid)} / {len(id2entity)}")

  1%|          | 47500/4818298 [06:50<10:50:13, 122.28it/s]

In [ ]:
with open('fid2qid.pkl', 'wb') as f:
    pickle.dump(fid2qid, f)

## Part 2: Convert Property IDs